In [6]:
# define the files in the repository
filePath          = 'https://raw.githubusercontent.com/Hotzkow/Testing-with-Way-Points/main/experiments/rq2-crossAppTestResults.csv'
robustnessRootDir = 'https://raw.githubusercontent.com/Hotzkow/Testing-with-Way-Points/main/experiments/prototype-results/rq3-explorations/robustnessResults-'
numTargetsList    = 'https://raw.githubusercontent.com/Hotzkow/Testing-with-Way-Points/main/experiments/rq1-baselineTargets.csv'
randomActions     = 'https://raw.githubusercontent.com/Hotzkow/Testing-with-Way-Points/main/experiments/rq1-actionTargetHistory-random.csv'
prototypeActions  = 'https://raw.githubusercontent.com/Hotzkow/Testing-with-Way-Points/main/experiments/rq1-actionTargetHistory-prototype.csv'


In [7]:
# in mybinder we have to install modules first
# !pip install pandas
# !pip install plotly>=4.0.0


# Finding Target Elements (RQ1)


In [8]:

# random needs at most 273 actions per scenario, for a better readable plot we limit the maximal number of actions displayed
limit = 300

from decimal import Decimal, getcontext, ROUND_HALF_UP

round_context = getcontext()
round_context.rounding = ROUND_HALF_UP

# the python default function `round` does not always round up at x.5 (e.g. round(2.5) -> 2)
def c_round(x, digits, precision=5):
    tmp = round(Decimal(x), precision)
    return float(tmp.__round__(digits))

# compute the table to visualize how many targets were located at any number of executed actions
def computeTable(df, maxval, maxIndex):
  table = pd.DataFrame(columns = ['category','scenario','actions','targets'])

  maxIndex =  min(maxIndex,limit)
  for scenario, data in df.groupby('scenario'):
  # scenario = 'b11'
  # data = df.groupby('scenario').get_group(scenario)
    last = 0
    appLast = {}
    for app in data['app']:
      appLast[app] = 0.0
    for i in range(0, maxIndex+1):
      entries = data[data['actions']==i]
      # print(entries)
      sumV = 0.0
      for app in entries['app']:
        e = entries['targets'][entries['app']==app].iloc[0]
        # print(f'{app}:{i}:{e}')
        appLast[app] = e
      for e in appLast.values():
        sumV = sumV + e
      # print(f'{appLast} -> sumV={sumV}')
      sumV = int(sumV) if (sumV % 1 == 0) else c_round(sumV,2)
      # print(sum)
      if (last!=sumV and sumV > 0.0):
        last=sumV
        # print(sumV)
        newRow = pd.Series([data['category'].iloc[0],data['scenario'].iloc[0],i,sumV], index=table.columns)
        
      else:
        newRow = pd.Series([data['category'].iloc[0],data['scenario'].iloc[0],i,last], index=table.columns)
      table = table.append(newRow, ignore_index=True)
    maxval.append(last)
  return table

In [9]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# read the table of data points into panda data object
dataTable = pd.read_csv(numTargetsList).applymap(lambda x: x.strip() if isinstance(x, str) else x)
numTargets = []
for scenario, data in dataTable.groupby('scenario'):
  numTargets.append(data['targets'].sum())
overallTargets = np.sum(numTargets)

print(f"number of targets: {numTargets}")  
# print(overallTargets)

pFile = pd.read_csv(prototypeActions).applymap(lambda x: x.strip() if isinstance(x, str) else x)
rFile = pd.read_csv(randomActions).applymap(lambda x: x.strip() if isinstance(x, str) else x)

maxval = []
protoMaxAction = pFile['actions'].max()
pTable = computeTable(pFile,maxval,protoMaxAction)
print(f"prototype targets found: {maxval}")
prototypeSum = c_round(np.sum(maxval),1)
maxrnd = []
rndMaxAction = rFile['actions'].max()
rTable = computeTable(rFile,maxrnd, rndMaxAction)
print(f"random targets found: {maxrnd}")
randomSum = c_round(np.sum(maxrnd),1)
print(f"prototype actions: {protoMaxAction}, prototype targets = {prototypeSum}, random actions: {rndMaxAction}, targets = {randomSum}")
sortbyActionsP = pFile.sort_values(['scenario','actions'],ascending=False).groupby('scenario').head(1).sort_values('scenario',ascending=True)
sortbyActionsR = rFile.sort_values(['scenario','actions'],ascending=False).groupby('scenario').head(1).sort_values('scenario',ascending=True)
# print(sortbyActionsP)

colors = ['#DDFFEE','#86FFDE ','#00EBC1','#00CBA7','#00AF8E', '#009175', '#00735C', '#005745', '#003D30', 'black']

def scenarioBar(s, table, idx):
  data = table.get_group(s)
  # texts = []
  # for e in data['targets']:
  #   texts.append(f"{s}:<br>{c_round(e,1)}")
  return go.Bar(name=s, x=data['actions'], y=data['targets'],marker_color=colors[idx], 
                marker_line_color= colors[idx],
                # text=data['targets'], #text=texts, 
                textposition='inside', textfont=dict(size=8),  showlegend=False)


number of targets: [5, 15, 16, 27, 44, 23, 24, 29, 13, 17]
prototype targets found: [5, 12.33, 15.89, 26.33, 42, 22.6, 16.9, 22.2, 12, 16]
random targets found: [5, 12.14, 12.9, 16.9, 13.95, 8, 7.84, 7.43, 12, 16.17]
prototype actions: 69, prototype targets = 191.3, random actions: 273, targets = 112.3


In [10]:
#@title

fig = make_subplots(rows=2,cols=2, 
                shared_xaxes=True, 
                # shared_yaxes=True, 
                column_widths=[0.85, 0.15],
                row_heights=[2/3,1/3],
                specs=[[{}, {"rowspan": 2}], [{}, None] ],
                vertical_spacing=0.08,
                horizontal_spacing=0.05,    
                subplot_titles=("Guido","Overall Results", "Random Exploration")
              )

# number of actions the prototype required to explore each scenario
pScenarios = pTable.groupby('scenario')
# number of actions the random exploration required to explore each scenario
rScenarios = rTable.groupby('scenario')

rndActions = 0.0
protoActions = 0.0
idx = 0
ticks=[]
greypalette=["rgb(0, 0, 0)","rgb(10,10,10)","rgb(20, 20, 20)","rgb(40, 40, 40)",
             "rgb(60, 60, 60)","rgb(80, 80, 80)","rgb(170, 170, 170)",
             "rgb(180, 180, 180)","rgb(200,200,200)","rgb(220,220,220)"]
# add all scenarios to graph as stacked bar plot
for scenario,data in pScenarios:
  
  fig.add_trace(scenarioBar(scenario,pScenarios, idx), row=1, col=1)
  
  #in scenario,sortedTable,actiontable,idx
  e=sortbyActionsP.iloc[idx]
  x = e['actions']
  if (x-1 not in ticks and x+1 not in ticks):  # too many ticks are not readable thus we have to space them out
    ticks.append(x)
  # select all scenarios which are drawn below this one to find the start point in y dimmension
  pystart = pTable[pTable.actions==x].iloc[:idx,]['targets'].sum()
  y2 = pystart+maxval[idx]
  fig.add_trace(go.Scatter(
      name = f'{scenario} completion', showlegend=False, x = [x, x],
      y = [pystart, y2], mode = 'lines',
      line = dict(color=greypalette[idx], dash='dot', width=1)
  ),row=1,col=1) 
  protoActions += x
  # print(f"{e['scenario']}({idx}) last action = {x}, avg targets = range({pystart},{y2})")

  er=sortbyActionsR.iloc[idx]
  x=er['actions']
  if (x-1 not in ticks and x+1 not in ticks):
    ticks.append(x)
  # select all scenarios which are drawn below this one to find the start point in y dimmension
  pystart = rTable[rTable.actions==x].iloc[:idx,]['targets'].sum()
  y2 = pystart+maxrnd[idx]
  fig.add_trace(go.Scatter(
      name = f'{scenario} rnd completion', showlegend=False, x = [x, x],
      y = [pystart, y2], mode = 'lines',
      line = dict(color=greypalette[idx], dash='dot', width=1)
  ),row=2,col=1) 
  rndActions += x
  # print(f"random {e['scenario']}({idx}) last action = {x}, avg targets = range({pystart},{y2})")

  fig.add_trace(scenarioBar(scenario,rScenarios, idx), row=2, col=1)
  fig.add_trace(go.Bar(name=scenario, x=[f"Overall Targets ({overallTargets})"], 
                       y=[numTargets[idx]], text=[numTargets[idx]], textposition='inside',marker_color=colors[idx], width=0.9),row=1,col=2)
  fig.add_trace(go.Bar( name=scenario, x=[f"Guido avg. ({prototypeSum})"], showlegend=False,
                       y=[maxval[idx]], text=[round(maxval[idx],1)], textposition='inside',marker_color=colors[idx], width=0.9),row=1,col=2)
  fig.add_trace(go.Bar( name=scenario, x=[f"Random avg. ({randomSum})"], showlegend=False,
                       y=[maxrnd[idx]], text=[round(maxrnd[idx],1)], textposition='inside',marker_color=colors[idx], width=0.9),row=1,col=2)
  idx = idx + 1

print(f"prototype actions = {protoActions}; random actions = {rndActions}")

# Change the bar mode
fig.update_layout(
    barmode='stack', height=800, width=1400,
    plot_bgcolor= "white",
    yaxis=dict(zerolinecolor = '#969696', title='Number of Correct Targets'),
    yaxis3=dict(zerolinecolor = '#969696', title='Number of Correct Targets'),
    xaxis=dict(zerolinecolor = '#969696'),
    xaxis1=dict(zerolinecolor = '#969696', tickvals = ticks, ticktext=ticks),
    xaxis3=dict(zerolinecolor = '#969696', title='Number of Actions', 
                ticks="inside", tickangle = 0, tickvals = ticks, ticktext=ticks, tickfont=dict(size=8)),
    legend=dict(
      yanchor="top",
      y=0.99,
      xanchor="right",
      x=1.006
    ),
    bargap=0, # gap between bars of adjacent location coordinates.
    # bargroupgap=0.1 # gap between bars of the same location coordinate.
    )
# print()
fig.show()

prototype actions = 323.0; random actions = 1441.0


# Cross App Testing Success (RQ2)

In [11]:
import pandas as pd
## this snippet computes a data frame based on the original csv at [filePath], 
# whereas group tuple of (app,precision,recall,simT) --but not by scenario right now-- into one entry and count how often they occured in the given csv data


# read the table of data points into panda data object
dataTable = pd.read_csv(filePath).sort_values(by='app')
dataTable = dataTable.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# print(data)
scaled = pd.DataFrame(columns=['app','category','scenario','simT','precision','recall','size'])

def numEntries(row,df):
  # [df['scenario']==row['scenario']]
  return df['app'][df['app']==row['app']][df['precision']==row['precision']][df['recall']==row['recall']][df['simT']==row['simT']].count()

for index, row in dataTable.iterrows():
  if numEntries(row,scaled)==0 :
    numTraces = dataTable['app'][dataTable['app']==row['app']][dataTable['simT']==row['simT']].count()
    size = numEntries(row,dataTable)/numTraces
    scaled = scaled.append(pd.Series([row['app'],row['category'],row['scenario'],row['simT'],row['precision'],row['recall'],size], index=scaled.columns), ignore_index=True)

# scaled

In [12]:
import statsmodels.api as sm

scale=1.5
similarities = [ 0.1, 0.15, 0.2, 0.25, 0.3 ]
# palette = ["#003C86", "#009FFA", "#008607", "#F60239", "#FFDC3D" ]
palette = ["rgb(0,114,178)","rgb(86,180,233)","rgb(0,158,115)","rgb(213,94,0)","rgb(240,228,66)"]
# add linear regression line for the given similarity value
# if sim==-1 the overall regression line will be drawn
def addTrendline(fig, dataframe, sim, row, col, name=None, showlegend=True, color=None):
  data = dataframe if (sim==-1) else dataframe.loc[dataframe['simT']==sim]
  prefix = 'trend' if (name is None) else name
  lineName = 'overall' if (sim==-1) else '$\\mathrm{{{0}}}\ \\mathrm{{sim}}_t = {1}$'.format(prefix, sim) 
  y = sm.OLS(data['recall'],sm.add_constant(data['precision'])).fit().fittedvalues
  
  if color is None:
    i = similarities.index(sim)
    fig.add_trace(go.Scatter(x=data['precision'], y=y,
                          mode = 'lines',
                          name=lineName,
                          marker=dict(size=4*scale/2, color=palette[i],), 
                          line=dict(width=1*scale, color=palette[i],
                                    # dash='dot' # dash options include 'dash', 'dot', and 'dashdot'
                          ), showlegend=showlegend,
                          # legendgroup=f'group{i+1}'
                        ), row=row, col=col)
  else:
    fig.add_trace(go.Scatter(x=data['precision'], y=y,
                          mode = 'lines+markers',
                          name=lineName,
                          marker=dict(size=4*scale/2, color=color),
                          line=dict(width=1*scale, color=color
                                    # dash='dot' # dash options include 'dash', 'dot', and 'dashdot'
                          ), showlegend=showlegend ), row=row, col=col)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [13]:
# draw scatter plot with one category per subfigure and each app in a different color

sim = 0.2
filtered = scaled.loc[scaled['simT'] == sim]
groups = filtered.groupby('category')

def addScatter(fig, inputdata, row, col, groupBy = 'app', colored=True):
  simGroups = inputdata.groupby('simT')
  for sim, data in simGroups:
    # print(sim)
    i = similarities.index(sim)
    # print(i)
    for item, data in data.groupby(groupBy):
      textarray = []
      for index, e in data.iterrows():
        textarray.append( f'{e["category"]}: {e["app"]}, {e["scenario"]} ({e["simT"]})' )
      # print(textarray)
        
      fig.add_trace( go.Scatter(
          x=data['precision'],
          y=data['recall'],
          name=rf'$\mathrm{{{item}}}$',
          text=data['app'] if groupBy=='app' else textarray,
          mode='markers',    
          marker_size=data['size'].tolist(),
          hovertemplate=
          "<b>%{text}</b><br><br>" +
          "Precision: %{x:}<br>" +
          "Recall: %{y:}<br>" +
          "Size: %{marker.size:%}" +
          "<extra></extra>",
          # legendgroup= "group" if (groupBy=='app') else f'group{i+1}',
          marker=dict(sizemode='area', sizeref=0.003 if groupBy=='app' else 0.01,
                      color=None if colored else palette[i],
                      line=dict(width=0.5, color='black') if colored else None),
          showlegend=colored
        ), row=row, col=col)
    


In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def renderDistribution():
  fig = make_subplots(rows=2,cols=3, shared_xaxes=True, vertical_spacing=0.05,
                      shared_yaxes=True, horizontal_spacing=0.01,
                      subplot_titles=("Varying Thresholds","Browser", "To Do List", "Shopping", "Mail Client", "Tip Calculator"))



  # iterating over categories (order is fixed by the given data/app names)
  categories = ["Browser", "To Do List", "Shopping", "Mail Client", "Tip Calculator"]
  idx = 1
  dataTableGroups=dataTable.groupby('category')
  for category in categories:  # FIXME wrong order with respecht to scatter plot
    # print(f'{scenario} : row = {1 if idx<3 else 2} col={(idx %3)+1}')
    row = 1 if idx<3 else 2
    col = (idx %3)+1
    categoryData = groups.get_group(category)
    addScatter(fig, categoryData, row, col)
    addTrendline(fig, dataTableGroups.get_group(category), sim, row, col, None, False, "grey")
    idx = idx + 1

  # print(dataTable)
  addTrendline(fig, dataTable, 0.1, 1, 1, "trend")
  addTrendline(fig, dataTable, 0.15, 1, 1, "trend")
  addTrendline(fig, dataTable, 0.2, 1, 1, "trend")
  addTrendline(fig, dataTable, 0.25, 1, 1, "trend")
  addTrendline(fig, dataTable, 0.30, 1, 1, "trend")
  # print overall scatter
  addScatter(fig,scaled,1,1,'category',False)

  # figure styling
  upperX = dict(gridcolor="lightgrey")
  lowerX = dict(gridcolor="lightgrey", title='Precision')
  midY = dict(gridcolor="lightgrey")
  lowerY = dict(gridcolor="lightgrey", zerolinecolor = '#969696')
  fig.update_layout(
      # color-disability friendly palette
    # colorway=["#b6dbff", "#006ddb","#b66dff", "#db6d00","#920000","#24ff24","#004949"],
    colorway=palette,
    width=800*scale,
    height=580*scale,
    margin=dict(
          l=0,
          r=0,
          b=100,
          t=100
      ),
    plot_bgcolor= "white",
    yaxis=dict(gridcolor="lightgrey", title='Recall'),
    yaxis2=midY, yaxis3=midY,
    yaxis4=dict(gridcolor="lightgrey", zerolinecolor = '#969696', title='Recall'),
    yaxis5=lowerY, yaxis6=lowerY,
    xaxis=dict(gridcolor="lightgrey", zerolinecolor = '#969696'),  xaxis2=upperX, xaxis3=upperX,
    xaxis4=dict(gridcolor="lightgrey", zerolinecolor = '#969696', title='Precision'), xaxis5=lowerX, xaxis6=lowerX,
    legend=dict(itemsizing='constant')
  )


  ticks = [20,40,60,80,100]
  # ticks = [10,20,30,40,50,60,70,80,90,100]
  fig.update_xaxes(tickvals=ticks,rangemode="tozero")
  fig.update_yaxes(tickvals=ticks,rangemode="tozero")
  fig.update_annotations(dict(font_size=16))
  # for col in [1, 2]:
  #     fig.add_annotation(dict(x=col / 2 - 0.4, y=0.8, xref="paper", yref="paper", 
  #                             text='trace %d' %col, showarrow=False))
  fig.show()

renderDistribution()

# Robustness of Testing with Way-Points (RQ3)

In [15]:
import pandas as pd
import plotly.graph_objects as go

colormap =  {"duplicate":"rgb(86,180,233)","swap":"rgb(0,158,115)", "delete":"rgb(213,94,0)"}

def drawMutations(rfig, app, showLegend=True):
  robustnessPath = f'{robustnessRootDir}{app}.csv'

  # read the table of data points into panda data object
  dataTable = pd.read_csv(robustnessPath)
  dataTable = dataTable.applymap(lambda x: x.strip() if isinstance(x, str) else x)

  for mutation, data in dataTable.groupby('subject'):
    testtargets = data['targets'].iloc[0]
    baseline = [mutation, 0.0, testtargets, testtargets, 100.0]
    data.loc[-1] = baseline # prepending a row
    data.index = data.index +1 #shifting index for prepend
    data = data.sort_index() # sorting by index

    rfig.add_annotation(x=data['mutations'].iloc[-1], y=data["found"].iloc[-1], 
                        text = app, font=dict(color=colormap[mutation]), arrowcolor=colormap[mutation] )
    rfig. add_trace(
        go.Scatter( showlegend=showLegend, legendgroup="group1" if (mutation=="delete") else "group2" if (mutation=="swap") else "group3",
            x=data["mutations"],
            y=data["found"],
            name=mutation,
            line = dict(dash='dash' if (mutation=="swap") else 'dot' if (mutation=="delete") else None,
                        color = colormap[mutation]),
            text = [app for i in data["mutations"]],
            hovertemplate = '<b>%{text}:</b><br><i>Mutations</i> = %{x}<br><i>Targets Found</i> = %{y}'
        )
    )


In [16]:
# show the relative impact of mutations to the number of targets found
from plotly.subplots import make_subplots

def drawReativeMutations(rfig, app, showLegend=False):
  robustnessPath = f'{robustnessRootDir}{app}.csv'

  # read the table of data points into panda data object
  dataTable = pd.read_csv(robustnessPath)
  dataTable = dataTable.applymap(lambda x: x.strip() if isinstance(x, str) else x)

  for mutation, data in dataTable.groupby('subject'):
    testtargets = data['targets'].iloc[0]
    baseline = [mutation, 0.0, testtargets, testtargets, 100.0]
    data.loc[-1] = baseline # prepending a row
    data.index = data.index +1 #shifting index for prepend
    data = data.sort_index() # sorting by index
    xaxis = data["mutations"].apply(lambda x: x/testtargets)
    yaxis = data["found"].apply(lambda x: x/testtargets)

    # rfig.add_annotation(x=xaxis.iloc[-1], y=yaxis.iloc[-1], 
                        # text = app, font=dict(color=colormap[mutation]), arrowcolor=colormap[mutation] )
    rfig. add_trace(
        go.Scatter( showlegend=showLegend, 
                   legendgroup="group1" if (mutation=="delete") else "group2" if (mutation=="swap") else "group3",
            x=xaxis,
            y=yaxis,
            mode = 'markers',
            # name=f'{app}-{mutation}',
            name=f'{mutation}',
            line = dict(dash='dash' if (mutation=="swap") else 'dot' if (mutation=="delete") else None,
                        color = colormap[mutation]),
            text = [app for i in xaxis],
            hovertemplate = '<b>%{text}:</b><br><i>Mutations</i> = %{x}<br><i>Targets Found</i> = %{y}'
        ), row=1, col=1 if (mutation=="delete") else 2 if (mutation=="swap") else 3
    )

rfig = make_subplots(rows=1,cols=3, vertical_spacing=0.05,
                    shared_yaxes=True, shared_xaxes=False, horizontal_spacing=0.02,
                    subplot_titles=("Delete","Duplicate", "Swap"))
drawReativeMutations(rfig,"a23",True)
drawReativeMutations(rfig,"a24")
drawReativeMutations(rfig,"a33")
drawReativeMutations(rfig,"a35")
drawReativeMutations(rfig,"a41")
drawReativeMutations(rfig,"a44")

midX = dict(tickformat=".2%", gridcolor="lightgrey", zerolinecolor = 'white', title='Relative Number of Mutations')
midY = dict(scaleanchor="x", scaleratio=1, tickformat=".2%", gridcolor="lightgrey", zerolinecolor = '#969696')
rfig.update_layout(
  width=1200,
  # height=500,
  plot_bgcolor= "white",
  yaxis=dict(scaleanchor="x", scaleratio=1, tickformat=".2%", gridcolor="lightgrey", zerolinecolor = '#969696', title='Relative Number of Correct Targets'),
  xaxis=dict(tickformat=".2%", gridcolor="lightgrey", zerolinecolor = '#969696', title='Relative Number of Mutations'),
  xaxis2=midX,
  xaxis3=midX,
  yaxis2 = midY,
  yaxis3 = midY
)

ticks = [0.20,0.40,0.60,0.80,1]
rfig.update_xaxes(tickvals=ticks, rangemode="tozero")
rfig.update_yaxes(tickvals=ticks, rangemode="tozero")


rfig.show()

In [ ]:
# to visualize each app separately in absolute numbers
debugFig = go.Figure()
drawMutations(debugFig,"a23",True)
drawMutations(debugFig,"a24")
drawMutations(debugFig,"a33")
drawMutations(debugFig,"a35")
drawMutations(debugFig,"a41")
drawMutations(debugFig,"a44")

debugFig.update_annotations(dict(
            xref="x",
            yref="y",
            showarrow=True,
            arrowhead=0,
            ax=10,
            ay=-10
))
debugFig.update_layout(
  width=600,
  height=600,
  plot_bgcolor= "white",
  yaxis=dict(gridcolor="lightgrey", zerolinecolor = '#969696', title='Number of Correct Targets'),
  xaxis=dict(gridcolor="lightgrey", zerolinecolor = '#969696', title='Number of Mutations'),
)
debugFig.update_xaxes(rangemode="tozero")
debugFig.update_yaxes(rangemode="tozero")

debugFig.show()

In [18]:
# store the file to google drive (when this notebook is executed via google colab)

# first we mount google drive to fetch our data file
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install plotly>=4.0.0
# !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
# !chmod +x /usr/local/bin/orca
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4

# rfig.write_image("/content/drive/My Drive/workspace/TwWPs/robustness.pdf")
# fig.write_image("/content/drive/My Drive/workspace/TwWPs/fig.pdf")